In [85]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [86]:
class OneToOneITU():
    def __init__(self, n, m, parameters = (None,None) , lbs=(0, 0)):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
     
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]
        
        self.max_A_j = np.max(parameters[0],0)

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, :]

    def get_V_ij(self, U_i, j_idx ,i_idx = None):
        if i_idx is None:
            return (self.B_ij[:,j_idx] -  U_i[:,None])/ self.A_ij[:,j_idx]
        else:
            return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [87]:
def generate_example(self, var_i, var_j, random_seed):
    np.random.seed(random_seed)
    k = 2

    θ_true = np.array([.2,-.2, 1,2])

    X_i = np.random.normal([5,var_i],[5,var_i], size = [self.n, k]).round(0)
    Y_j = np.random.normal([5,var_j],[5,var_j], size = [self.m, k]).round(0)
    #X_ijk = np.random.choice(10, size = (self.n_i,m_j, k))

    A = np.exp( np.abs((X_i[:,None,:]- Y_j[None,:,:]))@  θ_true[:k])
    B = np.abs((X_i[:,None,:]- Y_j[None,:,:])) @ θ_true[k:2*k] 
    print(len(np.unique(X_i)))
    self.A_ij = A
    self.B_ij = B
    self.max_A_j = np.max(A, axis= 0)

OneToOneITU.generate_example = generate_example

In [88]:
# def transpose_mkt(self):
#     n, m = self.n, self.m
#     self.m = n
#     self.n = m

#     self.A_ij = 1 / self.A_ij.T
#     self.B_ij = self.B_ij.T/ self.A_ij

# OneToOneITU.transpose_mkt = transpose_mkt

In [89]:
def check_primal_feas(self, mu_ij):
    print(f"i: { np.all(np.sum(mu_ij,axis=1) <= 1)}, j:  {np.all(np.sum(mu_ij,axis=0) <= 1) }")
    print(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}")

OneToOneITU.check_primal_feas = check_primal_feas

def check_IR(self,mu_ij,U_i,V_j, output = False):
    IR_i = np.sum((U_i - self.lb_U)* (1- np.sum(mu_ij,axis=1)))
    IR_j = np.sum((V_j - self.lb_V) * (1 - np.sum(mu_ij,axis=0))) 
    print(f"i: { IR_i}, j:  {IR_j }")
    if output is True:
        IR_i = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        IR_j = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return IR_i, IR_j
    
OneToOneITU.check_IR = check_IR

def check_CS(self,U_i,V_j, output = None):
    CS = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return CS, np.where(CS < 0 )
    print(f"min_ij D_ij : {(self.D_ij(U_i,V_j)).min()} ")
    
OneToOneITU.check_CS = check_CS 

def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    print("FEAS")
    self.check_primal_feas(mu_ij)
    print("________________")
    print("ε-CS")
    self.check_CS(U_i, V_j )
    print("________________")
    print("IR")
    self.check_IR(mu_ij, U_i, V_j )

OneToOneITU.check_all = check_all

# Auction algorithms

### Forward Auction

In [90]:
def forward_auction(self, data = None, tol_ε = None):
    if data is None:
        V_j = np.ones(self.m) * self.lb_V
        mu_i = np.ones( self.n, dtype= int) * (self.m +1)
    else:
        mu_ij_01 = data[0]
        V_j = data[1]
        id_i, id_j = np.where(mu_ij_01 > 0)
        mu_i = np.ones(self.n, dtype= int) * (self.m+1)
        mu_i[id_i] = id_j

    unassigned_i = np.where(mu_i == self.m + 1)[0] 
    n_unassigned_i = len(unassigned_i)
    iter = 0

    while n_unassigned_i > 0:
        iter += 1
        U_ij = self.get_U_ij( V_j, unassigned_i)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_i[unassigned_i] += (self.m - mu_i[unassigned_i]) * perm_unmatched
        
        else:
            ### bidding phase
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned_i,1))), axis = 1)
            j_i = np.argmax(U_ij, axis= 1)
            masked_U_ij = np.where(j_i[:,None] == np.arange(self.m+1)[None, :], -np.inf,U_ij)
            w_i = np.max(masked_U_ij, axis=1)
            j_i_unique = np.unique(j_i)
            offers = np.where(j_i[:,None] == j_i_unique[None,:], 
                              self.get_V_ij(w_i[:,None],j_i_unique[None,:],unassigned_i[:,None]), 
                              np.nan)
       
            ### assignment phase
            i_j_among_unass = np.nanargmax(offers, axis = 0)
            i_j = unassigned_i[ i_j_among_unass]
            best_offer_j = offers[i_j_among_unass, np.arange(len(j_i_unique))]

            # modify solution
            mu_i[  np.any( mu_i[:,None] == j_i_unique , axis = 1) ] = self.m +1
            mu_i[i_j] = j_i_unique
            
            V_j[j_i_unique] += best_offer_j - V_j[j_i_unique] + tol_ε/self.A_ij[i_j,j_i_unique]#np.maximum(best_offer_j - V_j[j_i_unique], tol_ε/self.A_ij[i_j,j_i_unique])
        
        unassigned_i = np.where(mu_i == self.m + 1)[0]
        n_unassigned_i = len(unassigned_i)
        
    print(f"for: {iter}")
    matched_i =  mu_i < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_i[matched_i]]
    mu_ij_01 = mu_i[:,None] == np.arange(self.m +1)

    return mu_ij_01[:,:-1], U_i, V_j
    

OneToOneITU.forward_auction = forward_auction

In [91]:
# mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = example_mkt.forward_auction(tol_ε= 1)
# print(np.all( u_i_noscaling[:,None]+ 1>=  example_mkt.get_U_ij(v_j_noscaling,np.arange(example_mkt.n)), axis= 1 ))
# np.min(u_i_noscaling[:,None] -  example_mkt.get_U_ij(v_j_noscaling,np.arange(example_mkt.n)))

#### TU

In [92]:
# n_TU = 25
# m_TU = 26
# A_TU = np.ones([n_TU,m_TU])
# B_TU = np.random.normal(40,5,[n_TU,m_TU]).round(0)

# example_TU = OneToOneITU( n_TU,m_TU, (A_TU,B_TU))
# mu_TU, u_TU, v_TU = example_TU.reverse_auction(tol_ε= 0.001)

In [93]:
# def solve_1to1(Φ_i_j):
#     n, m = np.shape(Φ_i_j)
#     M_z_a = spr.bmat([[spr.kron(spr.eye(n), np.ones((1, m)))],
#                       [spr.kron(np.ones((1, n)), spr.eye(m))]])
  
#     q = np.concatenate((np.ones(n), np.ones(m)))

#     model = grb.Model()
#     mu_a = model.addMVar(n * m, vtype=grb.GRB.INTEGER)
#     model.setObjective(Φ_i_j.flatten() @ mu_a, GRB.MAXIMIZE)
#     model.addConstr(M_z_a @ mu_a <= q)
    
#     model.Params.OutputFlag = 0
#     model.optimize()

#     return np.array(model.x, dtype= bool).reshape([n,m])*1

# mu_gurobi = solve_1to1(B_TU)
# print((mu_gurobi * B_TU).sum())
# print(u_TU.sum() +v_TU.sum())
# example_TU.check_all( (mu_TU, u_TU, v_TU ))

### Reverse Auction

In [94]:
def reverse_auction(self, data = None, tol_ε = None):
    if data is None:
        U_i = np.ones(self.n) * self.lb_U
        mu_j = np.ones(self.m, dtype= int) * (self.n +1)
    else:
        mu_j_01 = data[0]
        U_i = data[1]
        id_i, id_j = np.where(mu_j_01 >0)
        mu_j = np.ones(self.m, dtype= int) * (self.n+1)
        mu_j[id_j] = id_i

    unassigned_j = np.where(mu_j == self.n + 1)[0] 
    n_unassigned_j = len(unassigned_j)
    iter = 0

    while n_unassigned_j > 0:
        iter += 1
        V_ij = self.get_V_ij( U_i , unassigned_j)
        perm_unmatched = np.all(V_ij <= self.lb_V, axis = 0)
        
        if np.any(perm_unmatched):
            mu_j[unassigned_j] += (self.n - mu_j[unassigned_j]) * perm_unmatched
        
        else:
            ### bidding phase
            V_ij = np.concatenate((V_ij, self.lb_V * np.ones((1,n_unassigned_j))), axis = 0)
            i_j = np.argmax(V_ij, axis= 0)
            masked_V_ij = np.where( np.arange(self.n+1)[:, None] == i_j[None,:] , -np.inf,V_ij)
            β_j = np.max(masked_V_ij, axis=0)
            i_j_unique = np.unique(i_j)
            offers = np.where(i_j_unique[:,None] == i_j[None,:], 
                              self.get_U_ij(β_j,i_j_unique[:,None] ,unassigned_j[None,:]), 
                              np.nan)

            ### assignment phase
            j_i_among_unass = np.nanargmax(offers, axis = 1)
            j_i = unassigned_j[j_i_among_unass]
            best_offer_i = offers[np.arange(len(i_j_unique)),j_i_among_unass]
            # modify solution
            mu_j[ np.any( mu_j[:,None] == i_j_unique , axis = 1) ] = self.n +1
            mu_j[j_i] = i_j_unique
            U_i[i_j_unique] += best_offer_i - U_i[i_j_unique] + tol_ε * self.A_ij[i_j_unique,j_i]#np.maximum(best_offer_i - U_i[i_j_unique], tol_ε * self.A_ij[i_j_unique,j_i] )#/ self.max_A_j[j_i] )
                                        
        unassigned_j = np.where(mu_j == self.n + 1)[0]
        n_unassigned_j = len(unassigned_j)
        
    print(f"rev: {iter}") 
    matched_j =  mu_j < self.n
    V_j = np.ones(self.m) * self.lb_V
    V_j[mu_j < self.n] = self.get_V_ij(U_i, matched_j )[mu_j[matched_j],np.arange(matched_j.sum()) ]
    mu_j_01 = np.arange(self.n +1)[:,None] == mu_j[None,:]

    return mu_j_01[:-1,:], U_i, V_j
    

OneToOneITU.reverse_auction = reverse_auction

In [95]:
# n_i = 40
# m_j  = 30
# example_mkt = OneToOneITU( n_i,m_j)
# example_mkt.generate_example(2,2, random_seed = 1387)

In [96]:
# mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = example_mkt.reverse_auction(tol_ε= 1)
# print(np.min(v_j_noscaling[None,:] - example_mkt.get_V_ij(u_i_noscaling + 1, np.arange(example_mkt.m))))
# example_mkt.check_all( (mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling ))

### Forward-backward auction

In [97]:
def drop_for_scaling(self, mu_ij_01, U_i,V_j, tol_ε, side = 1):
    if side == 1:
        violations_j = np.any( V_j[None,:] + tol_ε <  self.get_V_ij(U_i  ,np.arange(self.m)), axis=  0)
        #violations_j = np.any( V_j[None,:] <  self.get_V_ij(U_i + tol_ε ,np.arange(self.m)), axis=  0)
        # print(violations_j.sum())
        mu_ij_01[:,violations_j] = 0

        print(f"dropped: {violations_j.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

      
    else:
        violations_i = np.any( U_i[:,None] + tol_ε <  self.get_U_ij(V_j  ,np.arange(self.n)), axis=  1)
   
        mu_ij_01[violations_i,:] = 0

        print(f"dropped: {violations_i.sum()}")
        # violations_i = np.any( U_i[:,None]+ tol_ε <  self.get_U_ij(V_j,np.arange(self.n)), axis=  1)
        # print(violations_i.sum())
        # mu_ij_01[violations_i,:] = 0

    return mu_ij_01, U_i,V_j
OneToOneITU.drop_for_scaling = drop_for_scaling

In [98]:
# n_i = 34
# m_j  = 30
# example_mkt = OneToOneITU( n_i,m_j)
# example_mkt.generate_example(2,2, random_seed = 19387)

In [99]:
# tol_ε = 5
# mu_ij_01, u_i, v_j = example_mkt.forward_auction(tol_ε= tol_ε)
# print(np.min(u_i[:,None] -  example_mkt.get_U_ij(v_j,np.arange(example_mkt.n))))

In [100]:
# tol_ε = tol_ε /2
# mu_ij_01, u_i, v_j  = example_mkt.drop_for_scaling(mu_ij_01, u_i, v_j ,tol_ε)
# mu_ij_01, u_i, v_j =  example_mkt.reverse_auction((mu_ij_01, u_i),tol_ε= tol_ε)
# print(np.min(v_j[None,:] - example_mkt.get_V_ij(u_i + tol_ε, np.arange(example_mkt.m))))

In [101]:
# mu_ij_01, u_i, v_j = example_mkt.forward_auction((mu_ij_01, v_j),tol_ε =tol_ε)
# print(mu_ij_01, u_i, v_j = example_mkt.forward_auction((mu_ij_01, v_j),tol_ε =tol_ε))

In [102]:
# example_mkt.check_all((mu_ij_01, u_i, v_j))

In [103]:
def forward_backward_scaling(self, tol_ε ,tol_ε_obj, scaling_factor):
    mu_ij_01, u_i, v_j = self.forward_auction(tol_ε = tol_ε)

    while tol_ε > tol_ε_obj:
        tol_ε *=  scaling_factor
        # print("####")
        # print( tol_ε)
        mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε)
        mu_ij_01, u_i, v_j = self.reverse_auction((mu_ij_01, u_i), tol_ε= tol_ε)
        # tol_ε *=  scaling_factor
        # mu_ij_01, u_i, v_j = self.drop_for_scaling(mu_ij_01, u_i, v_j , tol_ε, side = 0)
        mu_ij_01, u_i, v_j = self.forward_auction((mu_ij_01, v_j ), tol_ε = tol_ε)

        if np.all(self.check_CS(u_i,v_j,True)[0]>= - tol_ε_obj):
            return mu_ij_01, u_i, v_j

    return mu_ij_01, u_i, v_j
OneToOneITU.forward_backward_scaling =forward_backward_scaling  

In [118]:
n_i =  1000
m_j  = 1010
example_mkt = OneToOneITU( n_i,m_j)
example_mkt.generate_example(5,5, random_seed = 13099887)

31


In [119]:
example_mkt.B_ij.mean()

17.08238415841584

In [120]:
# mu_ij_01_noscaling, u_i_noscaling, v_j_noscaling = example_mkt.forward_auction(tol_ε= 1)

In [121]:
mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt.forward_backward_scaling(10,1e-11, 1/2)

for: 271
dropped: 876
rev: 450
for: 0
dropped: 720
rev: 173
for: 230
dropped: 593
rev: 911
for: 0
dropped: 530
rev: 357
for: 419
dropped: 497
rev: 775
for: 1872
dropped: 424
rev: 734
for: 832
dropped: 411
rev: 1148
for: 1492
dropped: 428
rev: 739
for: 1618
dropped: 431
rev: 1892
for: 0
dropped: 483
rev: 3866
for: 3709
dropped: 422
rev: 2844
for: 1694
dropped: 434
rev: 1426
for: 2393
dropped: 404
rev: 3315
for: 2295
dropped: 397
rev: 4302
for: 1593
dropped: 437
rev: 643
for: 2086
dropped: 393
rev: 1283
for: 3557
dropped: 411
rev: 3690
for: 0
dropped: 506
rev: 6264
for: 6743
dropped: 413
rev: 2023
for: 3616
dropped: 423
rev: 1881
for: 3800
dropped: 413
rev: 4119
for: 2304
dropped: 425
rev: 2593
for: 3615
dropped: 431
rev: 3020
for: 2508
dropped: 433
rev: 1275
for: 3657
dropped: 422
rev: 2564
for: 1974
dropped: 425
rev: 1955
for: 3288
dropped: 401
rev: 1309
for: 1686
dropped: 386
rev: 2692
for: 3783
dropped: 403
rev: 2644
for: 3884
dropped: 378
rev: 923
for: 2194
dropped: 397
rev: 1639
fo

In [122]:
example_mkt.check_all((mu_ij_01_scaling, u_i_scaling, v_j_scaling))

FEAS
i: True, j:  True
#matched: 1000 over 1000
________________
ε-CS
min_ij D_ij : -9.004435907018848e-12 
________________
IR
i: 0.0, j:  0.0


In [109]:
# n_i =  1000
# m_j  = 1020

# A_ = np.random.choice([.25,.5,1,2,4], size= [n_i,m_j])
# B_ = np.ones([n_i,m_j])*100000# np.random.choice([0,1], size= [n_i,m_j])*10000
# example_mkt_2 = OneToOneITU(  n_i,m_j,parameters=(A_,B_))

In [110]:
# mu_ij_01_noscaling2, u_i_noscaling2, v_j_noscaling2 = example_mkt_2.forward_auction(tol_ε= 1e-9)

In [111]:
# mu_ij_01_scaling, u_i_scaling, v_j_scaling = example_mkt_2.forward_backward_scaling(1,1e-9, 1/2)

In [112]:
# example_mkt_2.check_all((mu_ij_01_scaling, u_i_scaling, v_j_scaling))

To do:
* try nasty example with B binary times a constant
* try to scale also i forward auction
* try convex taxes

Next steps:
* $ij$-specific scaling
* max iter in the backward iteration
* (IMPORTANT) now all you are computing V_ij for all i and j in backward